## [dao_proposal.clvm]()
> This is a proposal which lives inside a singleton, it represents a proposed way to spend DAO money. If the proposal passes the INNERPUZ inside it will run, and the DAO Treasury will accept a spend to change itself or spend money

```clojire
(
  SINGLETON_STRUCT  ; (SINGLETON_MOD_HASH, (SINGLETON_ID, LAUNCHER_PUZZLE_HASH))
  PROPOSAL_MOD_HASH ; dao_proposal
  PROPOSAL_TIMER_MOD_HASH ; proposal timer needs to know which proposal created it, AND
  CAT_MOD_HASH ; CAT_V2
  TREASURY_MOD_HASH
  LOCKUP_MOD_HASH
  CAT_TAIL_HASH
  TREASURY_ID
  YES_VOTES  ; yes votes are +1, no votes don't tally - we compare yes_votes/total_votes at the end
  TOTAL_VOTES  ; how many people responded
  INNERPUZ  ; this is what runs if this proposal is successful
  vote_amount_or_solution  ; The qty of "votes" to add or subtract. ALWAYS POSITIVE.
  vote_info_or_p2_singleton_mod_hash
  vote_coin_id_or_current_cat_issuance  ; this is either the coin ID we're taking a vote from OR...
                                        ; the total number of CATs in circulation according to the treasury
  previous_votes  ; set this to 0 if we have passed
  lockup_innerpuzhash_or_attendance_required  ; this is either the innerpuz of the locked up CAT we're taking a vote from OR
                                              ; the attendance required - the percentage of the current issuance which must have voted represented as 0 to 10,000 - this is announced by the treasury
  pass_margin  ; this is what percentage of the total votes must be YES - represented as an integer from 0 to 10,000 - typically this is set at 5100 (51%)
  proposal_timelock  ; we assert this from the treasury and announce it, so the timer knows what the the current timelock is

)
```

- Singleton
- `SINGLETON_STRUCT`: (SINGLETON_MOD_HASH, (SINGLETON_ID, LAUNCHER_PUZZLE_HASH))
- `PROPOSAL_MOD_HASH`: dao_proposal
- `PROPOSAL_TIMER_MOD_HASH`: dao_proposal_timer
- `CAT_MOD_HASH`: cat_v2
- `TREASURY_MOD_HASH`: dao_treasury
- `LOCKUP_MOD_HASH`: dao_lockup
- `CAT_TAIL_HASH`: genesis_by_id?
- `TREASURY_ID`:
- `YES_VOTES`: yes votes are +1, no votes don't tally - we compare yes_votes/total_votes at the end
- `TOTAL_VOTES`: how many people responded
- `INNERPUZ`: this is what runs if this proposal is successful

- [test_dao_clvm.py]

In [7]:
from chia.wallet.puzzles.cat_loader import CAT_MOD

puzzle_path = "/Users/karlkim/dexie/chia-blockchain/chia/wallet/puzzles"
SINGLETON_MOD: Program = load_hex(f"{puzzle_path}/singleton_top_layer_v1_1.clvm.hex")
SINGLETON_LAUNCHER: Program = load_hex(f"{puzzle_path}/singleton_launcher.clvm.hex")
DAO_LOCKUP_MOD: Program = load_hex(f"{puzzle_path}/dao_lockup.clvm.hex")
DAO_PROPOSAL_TIMER_MOD: Program = load_hex(f"{puzzle_path}/dao_proposal_timer.clvm.hex")
DAO_PROPOSAL_MOD: Program = load_hex(f"{puzzle_path}/dao_proposal.clvm.hex")
DAO_TREASURY_MOD: Program = load_hex(f"{puzzle_path}/dao_treasury.clvm.hex")
P2_SINGLETON_MOD: Program = load_hex(f"{puzzle_path}/p2_singleton_or_delayed_puzhash.clvm.hex")
DAO_FINISHED_STATE: Program = load_hex(f"{puzzle_path}/dao_finished_state.clvm.hex")
DAO_RESALE_PREVENTION: Program = load_hex(f"{puzzle_path}/dao_resale_prevention_layer.clvm.hex")
DAO_CAT_TAIL: Program = load_hex(f"{puzzle_path}/genesis_by_coin_id_or_proposal.clvm.hex")
DAO_SPEND_PROPOSAL: Program = load_hex(f"{puzzle_path}/dao_spend_proposal.clvm.hex")

In [8]:
current_cat_issuance: uint64 = uint64(1000)
proposal_pass_percentage: uint64 = uint64(15)
CAT_TAIL: Program = Program.to("tail").get_tree_hash()
treasury_id: Program = Program.to("treasury").get_tree_hash()
LOCKUP_TIME: uint64 = uint64(200)
singleton_id: Program = Program.to("singleton_id").get_tree_hash()
singleton_struct: Program = Program.to(
    (SINGLETON_MOD.get_tree_hash(), (singleton_id, SINGLETON_LAUNCHER.get_tree_hash()))
)

In [9]:
full_proposal: Program = DAO_PROPOSAL_MOD.curry(
    singleton_struct,
    DAO_PROPOSAL_MOD.get_tree_hash(),
    DAO_PROPOSAL_TIMER_MOD.get_tree_hash(),
    CAT_MOD.get_tree_hash(),
    DAO_TREASURY_MOD.get_tree_hash(),
    DAO_LOCKUP_MOD.get_tree_hash(),
    CAT_TAIL,
    treasury_id,
    20,
    100,
    Program.to(1),
)

In [13]:
# Test Voting
voting_solution: Program = Program.to(
    [
        10, # vote_amount
        1,  # vote_info
        Program.to("vote_coin").get_tree_hash(), # vote coin id
        [0xFADEDDAB], # previous votes
        0xCAFEF00D, # lockup_innerpuzhash
    ]
)
conds: Program = full_proposal.run(solution)
print_program(conds)
print_conditions(
    full_proposal,
    solution
)

((62 0x8a33db157b86caf949c6661bd3a944b57801ae0e3651a6cd12ac1f3728ab995c) (63                                       
0xe4b2ab9273c426cb3da3fbaa38993a2cc36e6fd3bbf3a2f3ca6edb7d156cf370) (51                                            
0x520285ab5eb52b2786797a4bd290ed87951555175719a963b92a83bd1c1d4753 1                                               
(0x0bed8f7fad171680e1f0ad13377d8b7602c1bd917414b7110289306589b83537)))                                             

(CREATE_PUZZLE_ANNOUNCEMENT 0x8a33db157b86caf949c6661bd3a944b57801ae0e3651a6cd12ac1f3728ab995c)                    

(ASSERT_PUZZLE_ANNOUNCEMENT 0xe4b2ab9273c426cb3da3fbaa38993a2cc36e6fd3bbf3a2f3ca6edb7d156cf370)                    

(CREATE_COIN 0x520285ab5eb52b2786797a4bd290ed87951555175719a963b92a83bd1c1d4753 1)                                 

In [15]:
exit_solution = Program.to(
        [
            [[51, 0xCAFEF00D, 200]], # solution to inner_puz
            P2_SINGLETON_MOD.get_tree_hash(), # singleton_mod_hash
            current_cat_issuance,
            0, # previous votes
            LOCKUP_TIME,
            1000,
            5100,
            LOCKUP_TIME,
        ]
    )
conds: Program = full_proposal.run(exit_solution)
print_program(conds)
print_conditions(
    full_proposal,
    exit_solution
)

((51 0xfb015415f2e6a09c1141f880fc2135beec6adf2a19e4d02a191432846db16559 1 ("my_singleton_id")) (82 5) (63          
0x43cdaf328149fa93b68e30303c8cb28c36deb66d367df46ee66cb79d61efa670) (62                                            
0xd0f6e85befae86a4ab89374d8146b8b99104d6a19e850f9b976ec04739dff615) (63                                            
0xbf5f1d9606f594797fd068de0ad5b7714c693fece9182ed6e94211bcb486c500) (51 0x00cafef00d 200))                         

(CREATE_COIN 0xfb015415f2e6a09c1141f880fc2135beec6adf2a19e4d02a191432846db16559 1)                                 

(ASSERT_HEIGHT_RELATIVE 5)                                                                                         

(ASSERT_PUZZLE_ANNOUNCEMENT 0x43cdaf328149fa93b68e30303c8cb28c36deb66d367df46ee66cb79d61efa670)                    

(CREATE_PUZZLE_ANNOUNCEMENT 0xd0f6e85befae86a4ab89374d8146b8b99104d6a19e850f9b976ec04739dff615)                    

(ASSERT_PUZZLE_ANNOUNCEMENT 0xbf5f1d9606f594797fd068de0ad5b7714c693fece9182ed6e94211bcb486c500)                    

(CREATE_COIN 0x00cafef00d 200)                                                                                     

The workflow for DAO primitives seem to be the following:
1. Dexie create a DAO wallet (which also creates DAO CAT wallet and mint DAO Cat coins?) with treasury launcher id
https://github.com/Chia-Network/chia-blockchain/blob/021baba6c9e81e050742dc17c91b38c36329cea8/chia/wallet/dao_wallet/dao_wallet.py#L1199
    - DAO wallet is created
        - New DAO (DAO_Info) is created
        - DAO CAT coins are minted
        - Get DAO CAT tail id
        - 




1. Dexie creates a singleton Treasury Coin

https://github.com/Chia-Network/chia-blockchain/blob/3a2849fc59a81da5f6adb76cc85ed2f06cb8940b/chia/wallet/puzzles/dao_treasury.clvm#L4

2. Dexie creates a singleton DAO Proposal Coin

https://github.com/Chia-Network/chia-blockchain/blob/3a2849fc59a81da5f6adb76cc85ed2f06cb8940b/chia/wallet/puzzles/dao_proposal.clvm#L1


### DAO Wallet
> This is a wallet in the sense that it conforms to the interface needed by WalletStateManager. It is not a user-facing wallet. A user cannot spend or receive XCH though a wallet of this type. 

> It is expected that a wallet of type DAOCATWallet will be the user-facing wallet, and use a DAOWallet for state-tracking of the Treasury Singleton and its associated Proposals.



4. User creates a DAO CAT wallet with DAO CAT tail